In [2]:
import tensorflow as tf
import numpy as np

C:\Users\p\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
tf.reset_default_graph()

In [10]:
X_data = np.array([[[0.,1.]]])
batch_size = 1
hidden_size = 1

# None,1,2 는 무엇인가? 1 -> sequence length 2 -> vocab_size
X = tf.placeholder(tf.float32,[None,1,2])
X_data.shape 

(1, 1, 2)

In [12]:
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
# cell.shape =  (batch, hidden_size)
initial_state = cell.zero_state(batch_size,tf.float32)


In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    variables_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    for k,v in zip(variables_names,values):
        print(k,'\n',v)
    print('-'*9)
    outputs_val, state_val = sess.run([outputs,state],feed_dict={X:X_data})
    print('initial state:\n',sess.run(initial_state),
          '\nX_data val:\n',X_data,
          '\noutput val:\n',outputs_val,
          ']nstate val:\n',state_val)

rnn/basic_rnn_cell/kernel:0 
 [[-0.9973011 ]
 [-0.48289937]
 [ 0.38713038]]
rnn/basic_rnn_cell/bias:0 
 [0.]
---------
initial state:
 [[0.]] 
X_data val:
 [[[0. 1.]]] 
output val:
 [[[-0.44856262]]] ]nstate val:
 [[-0.44856262]]


In [67]:
tf.reset_default_graph()

In [68]:
data = 'hihello'
chars = list(set(data))
vocab_size= len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i,ch in enumerate(chars)}
X_data = [char_to_ix[ch] for ch in data]
X_onehot = tf.one_hot(X_data,vocab_size)
seq_length = len(data)-1
chars

['h', 'l', 'e', 'o', 'i']

In [69]:
hidden_size = 10 
batch_size = 1

In [70]:
def sample(x,n,cell,Why,by):
    W = cell[0]
    Wxh = W[:vocab_size]
    Whh = W[vocab_size:]
    bh = cell[1]
    h = np.zeros((1,hidden_size))
    ixes = []
    
    for step in range(n):
        h =np.tanh(np.dot(x,Wxh)+np.dot(h,Whh)+bh)
        y = np.dot(h,Why)+by
        ix = np.argmax(y)
        x = np.zeros((1,vocab_size))
        x[0][ix] = 1
        ixes.append(ix)
    return ixes
    

In [71]:
def seqtostr(input):
    return ''.join(ix_to_char[ch] for ch in input[:])

In [72]:
Why=tf.Variable(tf.random_normal([batch_size,hidden_size,vocab_size]))
by = tf.Variable(tf.random_normal([batch_size,1,vocab_size]),name='bias_y')

In [73]:
X=tf.placeholder(tf.float32,[None,1,vocab_size])
cell=tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size,tf.float32)
output,state = tf.nn.dynamic_rnn(cell=cell,inputs=X,initial_state=initial_state,dtype=tf.float32)

In [77]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    X_test = sess.run(X_onehot[0]).reshape(1,vocab_size)
    predtxt = sample(X_test,seq_length,sess.run(cell.variables),sess.run(Why),sess.run(by))
    print(data[0]+seqtostr(predtxt))

heheheh
